Importing packages

In [ ]:
using Plots
using XLSX
using YAML
using CSV
using DataFrames
using Statistics
using StatsBase
using Dates

Importing data

In [ ]:
# Read CSV files and convert to DataFrame
size_ARERA = "1_5_a_3"
#size_ARERA = "3_a_4_5"

# Define the file path
general_path = @__DIR__
# The original one was "../results/Dati/GSE_profiles_resident_v0_1_3_power.csv"
relative_path_GSE = "../results/Dati/GSE_profiles_resident_v1_Antrodoco.csv"
relative_path_GSE_2 = "../results/Dati/GSE_profiles_resident_v0_all_power.csv"
relative_path_GSE_3 = "../results/Dati/GSE_profiles_resident_v0_all_power_average.csv"
relative_path_ARERA = "../results/Dati/ARERA_profiles$size_ARERA.csv"
relative_path_Antrodoco = "../data/Antrodoco_average.csv"

file_GSE= joinpath(general_path,relative_path_GSE)
file_GSE_2= joinpath(general_path,relative_path_GSE_2)
file_GSE_3= joinpath(general_path,relative_path_GSE_3)
file_ARERA= joinpath(general_path,relative_path_ARERA)
file_Antrodoco= joinpath(general_path,relative_path_Antrodoco)

# Read data from the CSV
GSE_profiles= DataFrame(CSV.File(file_GSE))
GSE_profiles_2= DataFrame(CSV.File(file_GSE_2))
GSE_profiles_3= DataFrame(CSV.File(file_GSE_3))
ARERA_profiles= DataFrame(CSV.File(file_ARERA))
Antrodoco_profiles= DataFrame(CSV.File(file_Antrodoco))

# Substract 1 from "ora" column
Antrodoco_profiles.ora = Antrodoco_profiles.ora .- 1

# Show first 20 rows of all four DataFrame
first(Antrodoco_profiles, 20)

# Unite the dataframes

In [ ]:
# Create a new dataframe with the merge of the two dataframes
df_comparison = DataFrame(Mese = Int64[], Giorno = Int64[], Ora = Int64[], GSE_1_3_power = Float64[], GSE_all_power = Float64[], GSE_all_power_average = Float64[], ARERA = Float64[])

for i in 1:size(GSE_profiles,1)
    for j in 1:size(ARERA_profiles,1)
        if GSE_profiles[i,:Mese] == ARERA_profiles[j,:Mese] && GSE_profiles[i,:Giorno] == ARERA_profiles[j,:Giorno] && GSE_profiles[i,:Ora] == (ARERA_profiles[j,:Ora] - 1)
            push!(df_comparison, [GSE_profiles[i,:Mese], GSE_profiles[i,:Giorno], GSE_profiles[i,:Ora], 
            GSE_profiles[i,:PDMM], GSE_profiles_2[i,:PDMM], GSE_profiles_3[i,:PDMM], ARERA_profiles[j,:Valore_orario]])
        end
    end
end

# Aggiungi una nuova colonna al DataFrame
insertcols!(df_comparison, :Antrodoco => Antrodoco_profiles[:,"media profilo std"])

# Show the first 20 rows of the new DataFrame
first(df_comparison, 20)

# Calculate the consumption over the month for each profile and add it to a new dataframe called monthly consumption
monthly_consumption = DataFrame(Mese = Int64[], GSE_1_3_power = Float64[], GSE_all_power = Float64[], GSE_all_power_average = Float64[], ARERA = Float64[], Antrodoco = Float64[])
for i in 1:12
    push!(monthly_consumption, [i, sum(df_comparison[df_comparison.Mese .== i, :GSE_1_3_power]), sum(df_comparison[df_comparison.Mese .== i, :GSE_all_power]), sum(df_comparison[df_comparison.Mese .== i, :GSE_all_power_average]), sum(df_comparison[df_comparison.Mese .== i, :ARERA]), sum(df_comparison[df_comparison.Mese .== i, :Antrodoco])])
end

show(monthly_consumption, allcols=true)

# Plot

In [ ]:
# Show month 1 and month 6 of the new DataFrame
p0 = plot(xlab = "Ora", ylab = "Potenza [kW]", legend = :topleft)

#Selezione dei mesi e del giorno [21 o 22] per il plot
df_selected = df_comparison[((df_comparison.Mese .== 11)) .& (df_comparison.Giorno .== 10), :]

# Plotta i dati
i = unique(df_selected.Mese)
subset = df_selected[df_selected.Mese .== i, :]
plot!(p0, subset.Ora, subset.Antrodoco, label = "Antrodoco Mese $i", color = :blue, line = :solid)
plot!(p0, subset.Ora, subset.ARERA, label = "ARERA Mese $i", color = :black, line = :dash)
plot!(p0, subset.Ora, subset.GSE_1_3_power, label = "GSE Mese $i", color = :green, line = :dash)
plot!(p0, subset.Ora, subset.GSE_all_power, label = "GSE Mese $i", color = :red, line = :dash)

# Mostra il plot
display(p0)

# Salva il plot
savefig(joinpath(general_path,"../results/Immagini/Comparison_Mese_$i.png"))

Data analysis on MEAN and RMSE

In [ ]:
# Calculate the MAD (mean absolute difference) of the difference between the two profiles
mad1 = mean(abs.(df_comparison[!,"GSE_1_3_power"] .- df_comparison[!,"Antrodoco"]))
mad2 = mean(abs.(df_comparison[!,"GSE_all_power"] .- df_comparison[!,"Antrodoco"]))
mad3 = mean(abs.(df_comparison[!,"GSE_all_power_average"] .- df_comparison[!,"Antrodoco"]))
mad4 = mean(abs.(df_comparison[!,"ARERA"] .- df_comparison[!,"Antrodoco"]))

# Calcola il RMSE tra due vettori x e y
rmse1 = sqrt(mean((df_comparison[!,"GSE_1_3_power"] .- df_comparison[!,"Antrodoco"]).^2))
rmse2 = sqrt(mean((df_comparison[!,"GSE_all_power"] .- df_comparison[!,"Antrodoco"]).^2))
rmse3 = sqrt(mean((df_comparison[!,"GSE_all_power_average"] .- df_comparison[!,"Antrodoco"]).^2))
rmse4 = sqrt(mean((df_comparison[!,"ARERA"] .- df_comparison[!,"Antrodoco"]).^2))

# Arrotonda i risultati alla quinta cifra decimale e stampali
println("Mean Absolute Difference [kWh]: \n", "GSE 1-3 power: ", round(mad1, digits=5), "\nGSE all power: ", round(mad2, digits=5), "\nGSE all power average: ", round(mad3, digits=5), "\nARERA: ", round(mad4, digits=5))
println("Root Mean Squared Error [kWh]: \n", "GSE 1-3 power ", round(rmse1, digits=5), "\nGSE all power ", round(rmse2, digits=5), "\nGSE all power average ", round(rmse3, digits=5), "\nARERA ", round(rmse4, digits=5))


Comparison of hourly consumption during central hours to understand the sharing electricity between the three profiles

In [ ]:
# Filtra i dati per le ore centrali del giorno
df_central_hours = df_comparison[(df_comparison.Ora .>= 7) .& (df_comparison.Ora .<= 17), :]

# Raggruppa i dati per mese
groups = groupby(df_central_hours, :Mese)

# Calcola la somma delle potenze per ogni gruppo
df_sum = combine(groups, :GSE_1_3_power => sum => :F1_GSE_1_3_power, :GSE_all_power => sum => :F1_GSE_all_power, :GSE_all_power_average => sum => :F1_GSE_all_power_average, :ARERA => sum => :F1_ARERA, :Antrodoco => sum => :F1_Antrodoco)

# Mostra il DataFrame risultante
df_sum

# Calcola quanto si discostano i consumi medi mensili dalle ore centrali del giorno rispetto a F1_Antrodoco
mad1 = mean(abs.(df_sum[!,"F1_GSE_1_3_power"] .- df_sum[!,"F1_Antrodoco"]))
mad2 = mean(abs.(df_sum[!,"F1_GSE_all_power"] .- df_sum[!,"F1_Antrodoco"]))
mad3 = mean(abs.(df_sum[!,"F1_GSE_all_power_average"] .- df_sum[!,"F1_Antrodoco"]))
mad4 = mean(abs.(df_sum[!,"F1_ARERA"] .- df_sum[!,"F1_Antrodoco"]))
println("Mean Absolute Difference [kWh]: \n", "GSE 1-3 power: ", round(mad1, digits=5), "\nGSE all power: ", round(mad2, digits=5), "\nGSE all power average: ", round(mad3, digits=5), "\nARERA: ", round(mad4, digits=5))

# Calcola il RMSE tra due vettori x e y
rmse1 = sqrt(mean((df_sum[!,"F1_GSE_1_3_power"] .- df_sum[!,"F1_Antrodoco"]).^2))
rmse2 = sqrt(mean((df_sum[!,"F1_GSE_all_power"] .- df_sum[!,"F1_Antrodoco"]).^2))
rmse3 = sqrt(mean((df_sum[!,"F1_GSE_all_power_average"] .- df_sum[!,"F1_Antrodoco"]).^2))
rmse4 = sqrt(mean((df_sum[!,"F1_ARERA"] .- df_sum[!,"F1_Antrodoco"]).^2))
println("Root Mean Squared Error [kWh]: \n", "GSE 1-3 power ", round(rmse1, digits=5), "\nGSE all power ", round(rmse2, digits=5), "\nGSE all power average ", round(rmse3, digits=5), "\nARERA ", round(rmse4, digits=5))